<a href="https://colab.research.google.com/github/Gutobastos/imersao-iA-3-Alura/blob/main/projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
%pip -q install google-genai

Note: you may need to restart the kernel to use updated packages.
^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os

# E necessário inserir a chave na variável do seu ambiente com sua GOOGLE_API_KEY
google_api_key = os.environ.get('GOOGLE_API_KEY')
if google_api_key:
    print("Chave da API do Google encontrada nas variáveis de ambiente do seu PC.\n")
else:
    print("Chave da API do Google não encontrada nas variáveis de ambiente e tentaremos pelo GOOGLE COLAB.\n")
    #from google.colab import userdata
    #os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
from google import genai
client = genai.Client()
modelo = "gemini-2.0-flash"

In [10]:
%pip -q install google-adk

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 32] O arquivo já está sendo usado por outro processo: 'C:\\Users\\Fabricio-GNET\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python313\\site-packages\\authlib\\integrations\\base_client\\framework_integration.py'
Check the permissions.



In [ ]:
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

In [ ]:
from google.genai import types
chat_config = types.GenerateContentConfig(
    system_instruction = "Voce é um assistente pessoal sempre responda de forma educada e sucinta.",
)

prompt = input("Esperando a pergunta: ")
while prompt != "fim":
  chat = client.chats.create(model = modelo, config = chat_config)
  resposta = chat.send_message(prompt)
  print("Resposta: ", resposta.text, "\n")
  prompt = input("Esperando a pergunta ou escreva 'fim' para terminar: ")

print ("Fim do programa!")